https://www.janestreet.com/static/pdfs/puzzles/june-2025-puzzle.pdf

Scrolling in on the image, one can find Scrabble tiles along the table edges that read "PARTRIDGE TILING". That led me here: https://erich-friedman.github.io/mathmagic/0802.html

Based on this each table represents an incomplete partridge tiling of n=9 on a 45x45 grid.

Notably, the tile of size 1 is missing from each. This must be significant.

In [3]:
from ortools.sat.python import cp_model

We're gonna write each table as a constraint optimization problem, and use a SAT solver (OR-Tools) to solve them. This yields the location of our 1-tile.

I painstakingly copied each initial square into (size,x,y) format. I'll just use the top left table for this example:

In [4]:
initial_squares = [
    (3, 12, 4), # A square of size 3 at (12, 4)
    (4, 11, 0), (4, 33, 0), (4, 33, 4),
    (5, 9, 19), (5, 9, 24), (5, 33, 8),
    (6, 9, 13),
    (7, 31, 38), (7, 38, 8), (7, 38, 15), (7, 38, 38),
    (8, 9, 29), (8, 9, 37), (8, 29, 30), (8, 37, 0), (8, 37, 22), (8, 37, 30),
    (9, 0, 9), (9, 0, 18), (9, 0, 27), (9, 0, 36), (9, 15, 0), (9, 24, 0), (9, 29, 13)
]

Then set up our constraint model...

In [5]:
n = 9
grid_size = 45
model = cp_model.CpModel()

square_counts = {k: k for k in range(1, n + 1)}
for size, _, _ in initial_squares:
    square_counts[size] -= 1

x_vars = []
y_vars = []
sizes = []
x_intervals = []
y_intervals = []

for size in range(1, n + 1):
    count = square_counts[size]
    for i in range(count):
        x = model.NewIntVar(0, grid_size - size, f'x_{size}_{i}')
        y = model.NewIntVar(0, grid_size - size, f'y_{size}_{i}')
        x_vars.append(x)
        y_vars.append(y)
        sizes.append(size)

        x_interval = model.NewIntervalVar(x, size, x + size, f'x_int_{size}_{i}')
        y_interval = model.NewIntervalVar(y, size, y + size, f'y_int_{size}_{i}')
        x_intervals.append(x_interval)
        y_intervals.append(y_interval)

for i, (size, fx, fy) in enumerate(initial_squares):
    fx_const = model.NewConstant(fx)
    fy_const = model.NewConstant(fy)
    x_intervals.append(model.NewIntervalVar(fx_const, size, fx + size, f'x_fixed_{i}'))
    y_intervals.append(model.NewIntervalVar(fy_const, size, fy + size, f'y_fixed_{i}'))

model.AddNoOverlap2D(x_intervals, y_intervals)

With all of our constraints in place we can now solve. I've left out some code here that verifies uniqueness; status == cp_model.OPTIMAL is apparently not sufficient.

In [7]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
print(f"Solution was optimal? {status == cp_model.OPTIMAL}")

# The squares that were put down
new_squares = [(sizes[i], solver.Value(x_vars[i]), solver.Value(y_vars[i])) for i in range(len(sizes))]

# And the location of the 1 square..
one_square = [(x,y) for size,x,y in new_squares if size == 1][0]
print(f"{one_square=}") # one_square=(20,18)

Solution was optimal? True
one_square=(20, 18)


The final key to this puzzle was noticing the meaning of the gaps in between the tiles that spelled "PARTRIDGE TILING". It didn't hit me until I realized P was at x=16 (1 indexed...) and P is the 16th letter of the alphabet. So you can fill in A...ZA...A and so on along the top and left edges (note this overflows to the next table, it's global). Solving the remaining puzzles, and mapping them to the appropriate letter in this fashion, yields the final answer to the puzzle.

(20,18) maps to (U,S). I ended up with:

USOMCFBUSESIQSAUER

Huh? But when you try (y,x) instead of (x,y), you get:
SUMOFCUBESISSQUARE

Boom